In [1]:
import pandas as pd

import os
from pathlib import Path

# load functions from pixelator
from pixelator.pixeldataset import PixelDataset
from pixelator.graph import Graph

# load standard workflow single cell functions
from colocalization_measures.adjusted_local_assortativity import calculate_adjusted_local_assorativity
from colocalization_measures.colocalization_scores import create_colocalization_scores

# get directory with graph eval files
path_name = "C:/Users/janrh/Downloads/Pixelator_data"

In [2]:
# new graph for all data
sample_list = []
pxl_data_list = []

for index_place, file in enumerate(sorted(os.listdir(path_name))):
    
    # get name of current file in directory
    filename = os.fsdecode(file)

    # load marker edge list
    pxl_data = PixelDataset.from_file(path_name + "/" + filename)
    edge_list_df = pxl_data.edgelist   

    edge_list_df.columns = [col.lower() for col in edge_list_df.columns]

    # build the graph
    graph = Graph.from_edgelist(edgelist=edge_list_df,
                                add_marker_counts=True,
                                simplify=False,
                                use_full_bipartite=False)
    mega_graph = graph._raw
    
    # add filename as a vertex attribute
    mega_graph.es["sample"] = [filename] * mega_graph.ecount()

    sample_list.append(mega_graph)
    pxl_data_list.append(pxl_data)

    print(filename)

Uropod_CD54_fixed.dataset.pxl
Uropod_CD54_fixed_RANTES_stimulated.dataset.pxl
Uropod_control.dataset.pxl


In [3]:
# select sample to work on
sample_index = 2

# subgraphs of the selected sample
graph_component_subgraphs = sample_list[sample_index].components().subgraphs()

# load edge list from pixelator dataset
df = pxl_data_list[sample_index].edgelist

# create dictionary form all pixel names for each component
dict_comp_A = dict(zip(df["upia"].values, df["component"].values))
dict_comp_B = dict(zip(df["upib"].values, df["component"].values))

# create a set for all UPIA and UPIB for fast assessments
set_comp_A_pixels = set(df["upia"].values)
set_comp_B_pixels = set(df["upib"].values)

In [4]:
marker_names = ['CD9', 'CD62P', 'CD27', 'CD36', 'mIgG2b', 'CD337', 'CD3E', 'CD274', 'CD55', 'CD161', 'CD14', 'CD48', 'CD82', 'CD64', 
                'CD11c', 'CD54', 'CD11b', 'CD44', 'CD154', 'B2M', 'CD268', 'CD18', 'CD37', 'CD4', 'CD29', 'CD11a', 'CD47', 'CD7', 
                'CD2', 'CD35', 'CD45', 'CD8', 'CD314', 'CD22', 'CD19', 'CD127', 'CD53', 'CD52', 'CD229', 'CD72', 'CD59', 'CD163', 'CD38', 
                'CD25', 'CD41', 'CD150', 'CD278', 'ACTB', 'CD152', 'mIgG1',  'CD5', 'CD26', 'CD197', 'CD50', 'CD328', 'CD279', 'CD200',
                'CD71', 'CD102', 'CD244', 'CD45RB', 'CD40', 'CD45RA', 'CD84', 'CD49D', 'CD162', 'CD1d', 'CD137', 'CD32', 'CD69', 'CD20', 'CD33', 
                'CD158', 'HLA-ABC', 'mIgG2a', 'CD86', 'CD43', 'CD16', 'TCRb', 'HLA-DR']

In [5]:
# create new dataframe to keep track of similarity measures
similarity_measure_df = pd.DataFrame()
list_dfs = []

# set directory where the component df should be saved
directory_path = "C:/Users/janrh/Documents/GitHub/TDA-local-assortativity-paper/S0/"

# iterate through the first 100 components in the current sample
for component in graph_component_subgraphs:

    # get the component name by vertex name
    if component.vs[0]["name"] in set_comp_A_pixels:
        comp_name = dict_comp_A[component.vs[0]["name"]]
    else:
        comp_name = dict_comp_B[component.vs[0]["name"]]

    # get file info
    component_file_name = directory_path + str(comp_name)
    component_file_path = Path(component_file_name)

    # check if file in directry 
    if not component_file_path.is_file():

        # create a df with the local assorativity scores for the current component
        local_assort_score_df = pd.DataFrame()
        for marker in marker_names:

            # calculate local assorativity scores for each marker in list
            loc_rounded = calculate_adjusted_local_assorativity(component, marker)
            local_assort_score_df[marker] = loc_rounded   

        # save df
        local_assort_score_df.to_csv(component_file_name)

In [6]:
path_data = "C:/Users/janrh/Documents/GitHub/TDA-local-assortativity-paper/S1"

# calculate pairwise colocalization scores
similarity_measure_df_S1 = create_colocalization_scores(marker_names, path_data)

In [7]:
path_data = "C:/Users/janrh/Documents/GitHub/TDA-local-assortativity-paper/S3"

# calculate pairwise colocalization scores
similarity_measure_df_S3 = create_colocalization_scores(marker_names, path_data)